# Dataset Description

## awards_players

- playerId
- award (should standardize and abreviate)
- Lg ID (remove)
- year

## coaches

- lgId (remove)
- Stint (timeperiod coach was at the team) (use to convert years)
- Post_wins e Post_loses (playoff wins, irrelevant, remove)
  
## players_teams

- Stint (timeperiod player was in the team)
- lgId (remove)
- GP (games played)
- GS (game started)
- ORebounds (offensive rebounds)
- DRebounds (defensive rebounds)
- rebounds (total) (check sum of previous 2)
- turnovers (negative stat)
- PF (Personal Fouls)
- Fg Attempted (field goals attempted)
- Fg Made (maybe change to ratio)
- Ft (free throws)
- Dq (desqualify)
- Post ... (maybe ignore)

## Players

- POS (postion, has missing value) (Center, Forward-Center, Guard, Forward-Guard) (check if multiple positions over time)
- firstseason e lastseason (remove)
- height (missing values) (check if same units)
- college (some missing values, some none)
- death date (ignore dead players)

## series and teams post
- ignore files, reffering to the season after playoffs qualification  

## teams

- TmId (team Id), Franch ID (Franchise Id) (maybe irrelevant, maybe not)
- DivId (remove, all empty)
- Rank (maybe seed)
- Seeded (remove)
- **Playoff** (N, Y) to predict next
- first round, semis, finals (change to one column that has numbers that relate to when they got eliminated) (label0-no playoffs, label1-loss on first round, label2-loss on semis, label3-loss on finals, label4-no losses)
- Name (already have ID, may not be necessary)
- O_fgm (offensive field goals made)
- O_fga (offensive field goals attempted)
- Tm ORB, Tm DRB, Tm TRB ,Oppmt ORS, Oppmt DRB, Oppmt TRB (all 0, remove)
- GP (all teams play all games, maybe remove column)
- Win Loss -> create ratio
- Conf W L (maybe irrelevent)
- Attend (number of spectators, maybe irrelevant)
- Arena (remove)


# Developments

## players_teams

(All this was changed via python)
- Got per games for main statistical categories (Points, Assists, Rebounds (Off and Def), Steals, Blocks, Turnovers, Personal Fouls)
- Got percentages for main shots (Field Goals, Free Throws, Three Pointers)
- Removed League ID
- Created File with and without Post Season Performance
- Set all missing values to 0

![Alt text](prntscrns/ptms.png)


## awards_players

(All this was changed via python)
- Removed League ID
- Fixed an incorrectly inputted value (Kim Perrot Sportmanship -> Kim Perrot Sportmanship Award)
- Abbreviated the award names to a standard (Capitalized and First Letters Only)

![Alt text](prntscrns/awrds.png)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math

# teams_post data exploration and processing

In [ ]:
# TEAMS POST
# remove Lg ID
# check for missing values and abnormal data

teams_post = pd.read_csv("dataset/teams_post.csv")
teams_post.drop(columns=['lgID'], inplace=True)

missing_values = teams_post.isna().sum().sum()
print(f"Missing values: {missing_values}")

In [ ]:
for column in teams_post.columns:
    plt.figure(figsize=(8, 6))
    plt.scatter(range(len(teams_post)), teams_post[column], marker='.', label=column)
    plt.title(f'{column} Data Distribution')
    plt.ylabel(column)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
teams_post.to_csv('dataset_edited/teams_post.csv', index=False)

# coaches data exploration and processing

In [ ]:
# Coaches
# remove Lg ID
# check for missing values and abnormal data
# crate win/loss ratios

coaches = pd.read_csv("dataset/coaches.csv")
coaches.drop(columns=['lgID'], inplace=True)

#remove post data
coaches.drop(columns=['post_wins', 'post_losses'], inplace=True)

missing_values = teams_post.isna().sum().sum()
print(f"Missing values: {missing_values}")

In [ ]:
for column in coaches.columns:
    plt.figure(figsize=(8, 6))
    plt.scatter(range(len(coaches)), coaches[column], marker='.', label=column)
    plt.title(f'{column} Data Distribution')
    plt.ylabel(column)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# create w/l and post w/l ratio

coaches['W/L Ratio'] = coaches['won'] / (coaches['lost'].replace(0, 1))
#coaches['Post W/L Ratio'] = coaches['post_wins'] / (coaches['post_losses'].replace(0, 1))

coaches.to_csv('dataset_edited/coaches.csv', index=False)

# Players Data Exploration and processing

In [ ]:
# Players
# remove first and last season
# ignore dead players
# check for missing values and abnormal values

players = pd.read_csv("dataset/players.csv")
players.drop(columns=['firstseason', 'lastseason'], inplace=True)

In [ ]:
# List of columns to ignore for missing value check
columns_to_ignore = ["college", "collegeOther"]

# Find rows with missing values in all columns except the specified ones
missing_rows = players.dropna(subset=[col for col in players.columns if col not in columns_to_ignore])

missing_rows

In [ ]:
# remove dead player data
players_filtered = missing_rows[players['deathDate'] == '0000-00-00']
players_filtered.reset_index(drop=True, inplace=True)

players_filtered.drop(columns=['deathDate'], inplace=True)

players_filtered

In [ ]:
columns_filtered = [col for col in players_filtered.columns if col not in columns_to_ignore]
for column in columns_filtered:
    plt.figure(figsize=(8, 6))
    plt.scatter(range(len(players_filtered)), players_filtered[column], marker='.', label=column)
    plt.title(f'{column} Data Distribution')
    plt.ylabel(column)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# remove rows with birthdate 0000-00-00

players = players_filtered
players = players[players['birthDate'] != '0000-00-00']
players.reset_index(drop=True, inplace=True)

In [ ]:
# remove abnormal heights

players = players[players['height'] >= 50]

players.reset_index(drop=True, inplace=True)

In [ ]:
# remove abnormal weight

players = players[players['weight'] >= 50]

players.reset_index(drop=True, inplace=True)

In [ ]:
columns_filtered = [col for col in players.columns if col not in columns_to_ignore]
for column in columns_filtered:
    plt.figure(figsize=(8, 6))
    plt.scatter(range(len(players)), players[column], marker='.', label=column)
    plt.title(f'{column} Data Distribution')
    plt.ylabel(column)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
players.to_csv("dataset_edited/players.csv", index=False)
players

In [ ]:
teams_dataset = pd.read_csv("dataset/teams.csv")

def merge_column(row):
    if pd.isna(row['firstRound']) and pd.isna(row['semis']) and pd.isna(row['finals']):
        return 'label0'
    elif row['firstRound'] == 'L' and pd.isna(row['semis']) and pd.isna(row['finals']):
        return 'label1'
    elif row['firstRound'] == 'W' and row['semis'] == 'L' and pd.isna(row['finals']):
        return 'label2'
    elif row['firstRound'] == 'W' and row['semis'] == 'W' and row['finals'] == 'L':
        return 'label3'
    elif row['firstRound'] == 'W' and row['semis'] == 'W' and row['finals'] == 'W':
        return 'label4'
    else:
        return 'Unknown'
    
def merge_to_ratio(row):
    return round(row['won'] / row['GP'], 3)

def home_win_ratio(row):
    return round(row['homeW']/(row['homeW'] + row['homeL']), 3)

def away_win_ratio(row):
    return round(row['awayW']/(row['awayW'] + row['awayL']), 3)

teams_dataset.drop(['lgID', 'franchID', 'divID', 'seeded', 'name', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmTRB', 'opptmDRB', 'confL', 'confW', 'attend', 'arena'], axis = 1, inplace=True)
teams_dataset.rename(columns={'rank' : 'seed'}, inplace=True)
    
column_data = teams_dataset.apply(merge_column, axis=1)
teams_dataset.insert(4, 'results', column_data)
teams_dataset.drop(['firstRound', 'semis', 'finals'], axis=1, inplace = True)

column_data = teams_dataset.apply(merge_to_ratio, axis=1)
teams_dataset.insert(37, 'win_ratio', column_data)
teams_dataset.drop(['won', 'lost', 'GP'], axis=1, inplace=True)

column_data = teams_dataset.apply(home_win_ratio, axis=1)
teams_dataset.insert(38, 'homeW_ratio', column_data)
teams_dataset.drop(['homeW', 'homeL'], axis=1, inplace=True)

column_data = teams_dataset.apply(away_win_ratio, axis=1)
teams_dataset.insert(39, 'awayW_ratio', column_data)
teams_dataset.drop(['awayW', 'awayL'], axis=1, inplace=True)


for column in teams_dataset.iloc[:, 6:30]:
    plt.figure(figsize=(8, 6))
    plt.scatter(range(len(teams_dataset)), teams_dataset[column], marker='.', label=column)
    plt.title(f'{column} Data Distribution')
    plt.ylabel(column)
    plt.legend()
    plt.tight_layout()
    plt.show()



In [ ]:
seriesPost_dataset = pd.read_csv('dataset/series_post.csv')
seriesPost_dataset.drop(['lgIDWinner', 'lgIDLoser'], axis=1, inplace=True)

In [ ]:
import seaborn as sns 

teams_dataset.head()

# Select specific columns for correlation
selected_columns = ['o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_oreb', 'o_dreb', 'o_reb', 'o_stl', 'o_to', 'o_blk']

# Calculate the correlation between selected columns
correlation_matrix = teams_dataset[selected_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
teams_dataset['o_fg%'] = round(teams_dataset['o_fgm'] / teams_dataset['o_fga'] * 100, 2)
teams_dataset['o_ft%'] = round(teams_dataset['o_ftm'] / teams_dataset['o_fta'] * 100, 2)
teams_dataset['o_3p%'] = round(teams_dataset['o_3pm'] / teams_dataset['o_3pa'] * 100, 2)

teams_dataset['d_fg%'] = round(teams_dataset['d_fgm'] / teams_dataset['d_fga'] * 100, 2)
teams_dataset['d_ft%'] = round(teams_dataset['d_ftm'] / teams_dataset['d_fta'] * 100, 2)
teams_dataset['d_3p%'] = round(teams_dataset['d_3pm'] / teams_dataset['d_3pa'] * 100, 2)

teams_dataset.drop(['o_fgm', 'o_ftm', 'o_3pm'], axis=1, inplace=True)

teams_dataset.head()